In [0]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Data_Science_in_Health_Care/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/Data_Science_in_Health_Care


In [0]:
import pandas as pd
import re

# 1. Prepare Practice Data File

## 1.1 Read base file

In [5]:
bnf_codes = pd.read_csv("./data/20200311_1583928188440_BNF_Code_Information.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,11,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
bnf_codes.columns

Index(['BNF Chapter', 'BNF Chapter Code', 'BNF Section', 'BNF Section Code',
       'BNF Paragraph', 'BNF Paragraph Code', 'BNF Subparagraph',
       'BNF Subparagraph Code', 'BNF Chemical Substance',
       'BNF Chemical Substance Code', 'BNF Product', 'BNF Product Code',
       'BNF Presentation', 'BNF Presentation Code'],
      dtype='object')

In [7]:
bnf_codes["BNF Subparagraph Code"].unique

<bound method Series.unique of 0         101010
1         101010
2         101010
3         101010
4         101010
          ...   
81650    2398585
81651    2398585
81652    2398585
81653    2398585
81654    2399000
Name: BNF Subparagraph Code, Length: 81655, dtype: int64>

In [8]:
drug_sales = pd.read_csv("./data/data_files_2018_07/T201807PDPI BNFT.CSV")

# Gewisse Spaltennamen haben Tabs oder Leerzeichen am Ende -> entfernen
drug_sales_columns = {}
for name in drug_sales.columns:
  drug_sales_columns.update({name: name.strip()}   
  )
drug_sales.rename(columns=drug_sales_columns, inplace=True)
drug_sales

,SHA,PCT,PRACTICE,BNF CODE,BNF NAME,ITEMS,NIC,ACT COST,QUANTITY,PERIOD,
0,Q44,RTV,Y04937,0204000R0AAAHAH,Propranolol HCl_Tab 10mg,1,7.24,6.73,56,201807,
1,Q44,RTV,Y04937,0304010W0AAALAL,Promethazine HCl_Tab 25mg,1,2.33,2.28,28,201807,
2,Q44,RTV,Y04937,0401010Y0AAABAB,Zolpidem Tart_Tab 10mg,1,0.40,0.48,14,201807,
3,Q44,RTV,Y04937,0401010Z0AAAAAA,Zopiclone_Tab 7.5mg,3,0.76,1.05,28,201807,
4,Q44,RTV,Y04937,0401020B0AAAAAA,Buspirone HCl_Tab 5mg,1,3.59,3.44,28,201807,
...,...,...,...,...,...,...,...,...,...,...,...
9751486,Q70,99M,Y04634,1001010N0AAADAD,Mefenamic Acid_Tab 500mg,1,92.79,86.24,100,201807,
9751487,Q70,99M,Y04634,1304000D0BCAABB,Diprosalic_Oint 0.05%/3%,1,3.18,2.96,30,201807,
9751488,Q70,99M,Y04634,1304000G0AABABA,Clobetasol Prop_Oint 0.05%,1,7.90,7.35,100,201807,
9751489,Q70,99M,Y04634,21220000100,Epaderm Oint,2,13.06,12.15,1000,201807,


## 1.2 Read BNF Code to single Columns

In [0]:
# BNF Code: 01 ...
drug_sales["BNF Chapter Code"] = drug_sales["BNF CODE"].apply(lambda x: int(re.findall(r"^[0-9]{2}",x)[0]))

In [0]:
drug_sales["BNF Section Code"] = drug_sales["BNF CODE"].apply(lambda x: int(re.findall(r"^[0-9]{4}",x)[0]))

In [0]:
drug_sales["BNF Paragraph Code"] = drug_sales["BNF CODE"].apply(lambda x: int(re.findall(r"^[0-9]{6}",x)[0]))

In [0]:
drug_sales["BNF Subparagraph  Code"] = drug_sales["BNF CODE"].apply(lambda x: int(re.findall(r"^[0-9]{7}",x)[0]))

In [13]:
drug_sales_by_chapter = drug_sales[["PRACTICE", "BNF Chapter Code", "ITEMS", "NIC"]].groupby(["PRACTICE", "BNF Chapter Code"]).agg("sum")
drug_sales_by_chapter

ITEMS      NIC
PRACTICE BNF Chapter Code                
A81001   1                   691  3280.54
         2                  1995  9203.19
         3                   586  8328.43
         4                  1980  9774.12
         5                   241   981.37
...                          ...      ...
Y06192   11                    2    30.96
         12                    8    27.57
         13                    4    10.71
         21                    1     5.26
Y06209   6                     1    14.40

[157080 rows x 2 columns]

## 1.3 Aggregate Data per Practice (Chapter)

### Aggregate Data

In [14]:
drug_sales_sumemd_by_practice = drug_sales[["PRACTICE", "ITEMS", "NIC"]].groupby("PRACTICE").agg("sum")
drug_sales_sumemd_by_practice

,ITEMS,NIC
PRACTICE,,
A81001,8005,58349.07
A81002,38669,291568.90
A81004,18514,156655.28
A81005,16639,110988.46
A81006,28890,203691.89
...,...,...
Y06150,8,93.70
Y06168,3,17.29
Y06178,1,1.81


In [15]:
drug_sales_by_chapter = drug_sales[["PRACTICE", "BNF Chapter Code", "ITEMS", "NIC"]].groupby(["PRACTICE", "BNF Chapter Code"]).agg("sum")
drug_sales_by_chapter

ITEMS      NIC
PRACTICE BNF Chapter Code                
A81001   1                   691  3280.54
         2                  1995  9203.19
         3                   586  8328.43
         4                  1980  9774.12
         5                   241   981.37
...                          ...      ...
Y06192   11                    2    30.96
         12                    8    27.57
         13                    4    10.71
         21                    1     5.26
Y06209   6                     1    14.40

[157080 rows x 2 columns]

### Normalize Data

In [0]:
drug_sales_by_chapter_normalized = drug_sales_by_chapter.copy()
drug_sales_by_chapter_normalized["ITEMS"] = drug_sales_by_chapter["ITEMS"] / drug_sales_sumemd_by_practice["ITEMS"]
drug_sales_by_chapter_normalized["NIC"] = drug_sales_by_chapter["NIC"] / drug_sales_sumemd_by_practice["NIC"]
drug_sales_by_chapter_normalized.reset_index(inplace=True)

In [17]:
drug_sales_by_chapter_normalized

,PRACTICE,BNF Chapter Code,ITEMS,NIC
0,A81001,1,0.086321,0.056223
1,A81001,2,0.249219,0.157726
2,A81001,3,0.073204,0.142735
3,A81001,4,0.247345,0.167511
4,A81001,5,0.030106,0.016819
...,...,...,...,...
157075,Y06192,11,0.025000,0.082739
157076,Y06192,12,0.100000,0.073679
157077,Y06192,13,0.050000,0.028622
157078,Y06192,21,0.012500,0.014057


### Chapters to Columns

In [0]:
# Create Pandas DF with ITEMS and all Chapter Codes in column names 
drug_sales_by_chapter_pivot_only_items = drug_sales_by_chapter_normalized.pivot(index= 'PRACTICE', columns='BNF Chapter Code', values=["ITEMS"])
drug_sales_by_chapter_pivot_only_items.fillna(0, inplace=True)
drug_sales_by_chapter_pivot_only_items = drug_sales_by_chapter_pivot_only_items.droplevel(level=0, axis=1)
col_names = {}
for col in drug_sales_by_chapter_pivot_only_items.columns:
  col_names.update(
      {col : f"ITEMS_{col}"}
  )
drug_sales_by_chapter_pivot_only_items.rename(columns=col_names, inplace=True)

# Create Pandas DF with NIC and all Chapter Codes in column names 
drug_sales_by_chapter_pivot_only_nic = drug_sales_by_chapter_normalized.pivot(index= 'PRACTICE', columns='BNF Chapter Code', values="NIC")
drug_sales_by_chapter_pivot_only_nic.fillna(0, inplace=True)
#drug_sales_by_chapter_pivot_only_nic = drug_sales_by_chapter_pivot_only_nic.droplevel(level=0, axis=1)
col_names = {}
for col in drug_sales_by_chapter_pivot_only_nic.columns:
  col_names.update(
      {col : f"NIC_{col}"}
  )
drug_sales_by_chapter_pivot_only_nic.rename(columns=col_names, inplace=True)

In [57]:
# JOIN the two DF back together
#drug_sales_by_chapter_pivot_only_nic.index
drug_sales_by_chapters = drug_sales_by_chapter_pivot_only_nic.merge(drug_sales_by_chapter_pivot_only_items, left_index=True, right_index=True)
drug_sales_by_chapters

BNF Chapter Code,NIC_1,NIC_2,NIC_3,NIC_4,NIC_5,NIC_6,NIC_7,NIC_8,NIC_9,NIC_10,NIC_11,NIC_12,NIC_13,NIC_14,NIC_15,NIC_18,NIC_19,NIC_20,NIC_21,NIC_22,NIC_23,ITEMS_1,ITEMS_2,ITEMS_3,ITEMS_4,ITEMS_5,ITEMS_6,ITEMS_7,ITEMS_8,ITEMS_9,ITEMS_10,ITEMS_11,ITEMS_12,ITEMS_13,ITEMS_14,ITEMS_15,ITEMS_18,ITEMS_19,ITEMS_20,ITEMS_21,ITEMS_22,ITEMS_23
PRACTICE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A81001,0.056223,0.157726,0.142735,0.167511,0.016819,0.156060,0.037295,0.017994,0.074164,0.011891,0.027607,0.007233,0.037510,0.009665,0.007483,0.0,0.001373,0.015130,0.031614,0.004434,0.019533,0.086321,0.249219,0.073204,0.247345,0.030106,0.096190,0.020862,0.004997,0.061212,0.022986,0.019738,0.008245,0.041099,0.004747,0.000750,0.0,0.001124,0.007995,0.018863,0.001374,0.003623
A81002,0.061945,0.145536,0.161891,0.157818,0.018375,0.149729,0.034502,0.021213,0.077802,0.014817,0.022739,0.006790,0.033317,0.004202,0.003055,0.0,0.001670,0.012934,0.037903,0.003074,0.030689,0.094262,0.293181,0.078331,0.209160,0.027412,0.089865,0.024542,0.003491,0.062686,0.029300,0.017818,0.008715,0.027878,0.002198,0.000905,0.0,0.000905,0.005120,0.018490,0.001060,0.004681
A81004,0.060667,0.143918,0.110334,0.169418,0.031567,0.148483,0.042506,0.013335,0.077945,0.012791,0.020668,0.006895,0.040052,0.000800,0.003071,0.0,0.001653,0.028635,0.043309,0.003559,0.040393,0.086421,0.288862,0.065302,0.211678,0.032192,0.108945,0.027115,0.004267,0.040834,0.029977,0.018797,0.010263,0.036621,0.000486,0.000756,0.0,0.001188,0.007994,0.021659,0.001188,0.005455
A81005,0.073851,0.163154,0.134273,0.135219,0.016623,0.130996,0.047607,0.025108,0.055085,0.010108,0.021823,0.008470,0.035835,0.003579,0.003217,0.0,0.001429,0.017893,0.064823,0.006161,0.044748,0.098744,0.345634,0.062744,0.161849,0.023619,0.108961,0.033415,0.005770,0.042551,0.025242,0.018571,0.011659,0.025422,0.001683,0.000901,0.0,0.000841,0.003846,0.021275,0.001683,0.005589
A81006,0.058561,0.134753,0.184717,0.153028,0.022751,0.139594,0.035716,0.031853,0.071647,0.015066,0.019302,0.007284,0.034108,0.001410,0.005415,0.0,0.003071,0.020939,0.040715,0.002512,0.017556,0.094323,0.309311,0.088266,0.204500,0.026653,0.096435,0.022015,0.003565,0.042575,0.027449,0.013776,0.008169,0.028072,0.000658,0.001142,0.0,0.000935,0.007269,0.020249,0.000865,0.003773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y06150,0.000000,0.000000,0.183671,0.092529,0.000000,0.106617,0.000000,0.000000,0.110032,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.015155,0.000000,0.491996,0.000000,0.000000,0.125000,0.250000,0.000000,0.125000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.125000,0.000000,0.125000
Y06168,0.009832,0.990168,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
Y06178,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


### Write Data to csv

In [0]:
drug_sales_by_chapters.to_csv('./data_mod/bnf_data_by_practice_and_chapters.csv')

## 1.4 Aggregate Data per Practice (Section)

### Aggregate Data

In [40]:
drug_sales_sumemd_by_practice = drug_sales[["PRACTICE", "ITEMS", "NIC"]].groupby("PRACTICE").agg("sum")
drug_sales_sumemd_by_practice

,ITEMS,NIC
PRACTICE,,
A81001,8005,58349.07
A81002,38669,291568.90
A81004,18514,156655.28
A81005,16639,110988.46
A81006,28890,203691.89
...,...,...
Y06150,8,93.70
Y06168,3,17.29
Y06178,1,1.81


In [41]:
drug_sales_by_section = drug_sales[["PRACTICE", "BNF Section Code", "ITEMS", "NIC"]].groupby(["PRACTICE", "BNF Section Code"]).agg("sum")
drug_sales_by_section

ITEMS     NIC
PRACTICE BNF Section Code               
A81001   101                  44  217.27
         102                  28  200.29
         103                 407  803.14
         104                  12   36.90
         105                  23  834.06
...                          ...     ...
Y06192   1202                  2    7.95
         1304                  3    9.29
         1310                  1    1.42
         2122                  1    5.26
Y06209   604                   1   14.40

[856968 rows x 2 columns]

### Normalize Data

In [0]:
drug_sales_by_section_normalized = drug_sales_by_section.copy()
drug_sales_by_section_normalized["ITEMS"] = drug_sales_by_section["ITEMS"] / drug_sales_sumemd_by_practice["ITEMS"]
drug_sales_by_section_normalized["NIC"] = drug_sales_by_section["NIC"] / drug_sales_sumemd_by_practice["NIC"]
drug_sales_by_section_normalized.reset_index(inplace=True)

In [43]:
drug_sales_by_section_normalized

,PRACTICE,BNF Section Code,ITEMS,NIC
0,A81001,101,0.005497,0.003724
1,A81001,102,0.003498,0.003433
2,A81001,103,0.050843,0.013764
3,A81001,104,0.001499,0.000632
4,A81001,105,0.002873,0.014294
...,...,...,...,...
856963,Y06192,1202,0.025000,0.021246
856964,Y06192,1304,0.037500,0.024827
856965,Y06192,1310,0.012500,0.003795
856966,Y06192,2122,0.012500,0.014057


### Chapters to Columns

In [0]:
# Create Pandas DF with ITEMS and all Section Codes in column names 
drug_sales_by_section_pivot_only_items = drug_sales_by_section_normalized.pivot(index= 'PRACTICE', columns='BNF Section Code', values=["ITEMS"])
drug_sales_by_section_pivot_only_items.fillna(0, inplace=True)
drug_sales_by_section_pivot_only_items = drug_sales_by_section_pivot_only_items.droplevel(level=0, axis=1)
col_names = {}
for col in drug_sales_by_section_pivot_only_items.columns:
  col_names.update(
      {col : f"ITEMS_{col}"}
  )
drug_sales_by_section_pivot_only_items.rename(columns=col_names, inplace=True)

# Create Pandas DF with NIC and all Sections Codes in column names 
drug_sales_by_section_pivot_only_nic = drug_sales_by_section_normalized.pivot(index= 'PRACTICE', columns='BNF Section Code', values="NIC")
drug_sales_by_section_pivot_only_nic.fillna(0, inplace=True)
#drug_sales_by_section_pivot_only_nic = drug_sales_by_section_pivot_only_nic.droplevel(level=0, axis=1)
col_names = {}
for col in drug_sales_by_section_pivot_only_nic.columns:
  col_names.update(
      {col : f"NIC_{col}"}
  )
drug_sales_by_section_pivot_only_nic.rename(columns=col_names, inplace=True)

In [59]:
# JOIN the two DF back together
#drug_sales_by_section_pivot_only_nic.index
drug_sales_by_sections = drug_sales_by_section_pivot_only_nic.merge(drug_sales_by_section_pivot_only_items, left_index=True, right_index=True)
drug_sales_by_sections

BNF Section Code,NIC_101,NIC_102,NIC_103,NIC_104,NIC_105,NIC_106,NIC_107,NIC_108,NIC_109,NIC_201,NIC_202,NIC_203,NIC_204,NIC_205,NIC_206,NIC_207,NIC_208,NIC_209,NIC_211,NIC_212,NIC_301,NIC_302,NIC_303,NIC_304,NIC_307,NIC_308,NIC_309,NIC_310,NIC_311,NIC_401,NIC_402,NIC_403,NIC_404,NIC_405,NIC_406,NIC_407,NIC_408,NIC_409,NIC_410,NIC_411,...,ITEMS_2140,ITEMS_2141,ITEMS_2142,ITEMS_2143,ITEMS_2146,ITEMS_2147,ITEMS_2148,ITEMS_2149,ITEMS_2202,ITEMS_2205,ITEMS_2210,ITEMS_2215,ITEMS_2230,ITEMS_2240,ITEMS_2250,ITEMS_2260,ITEMS_2270,ITEMS_2280,ITEMS_2285,ITEMS_2290,ITEMS_2305,ITEMS_2310,ITEMS_2315,ITEMS_2320,ITEMS_2325,ITEMS_2330,ITEMS_2335,ITEMS_2345,ITEMS_2346,ITEMS_2350,ITEMS_2355,ITEMS_2360,ITEMS_2370,ITEMS_2375,ITEMS_2380,ITEMS_2385,ITEMS_2390,ITEMS_2392,ITEMS_2394,ITEMS_2396
PRACTICE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A81001,0.003724,0.003433,0.013764,0.000632,0.014294,0.012271,0.000506,0.0,0.007597,0.000615,0.004754,0.002398,0.008763,0.017307,0.018635,0.0,0.076183,0.008681,0.000826,0.019565,0.057410,0.073534,0.000802,0.008215,0.002705,0.0,0.000069,0.0,0.0,0.002528,0.012575,0.022171,0.002090,0.002999,0.001948,0.057078,0.055202,0.003307,0.000000,0.007612,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000125,0.000125,0.000625,0.000500,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000500,0.0,0.00000,0.000000,0.000375,0.000125,0.000000,0.000000,0.0,0.000250,0.000000,0.000000,0.002374,0.000000,0.000000,0.00000,0.000000,0.000000
A81002,0.002171,0.004811,0.012788,0.000500,0.013234,0.014250,0.000717,0.0,0.013473,0.000435,0.004296,0.000164,0.009288,0.021974,0.026968,0.0,0.056879,0.007467,0.000273,0.017790,0.060676,0.088725,0.000868,0.007562,0.004043,0.0,0.000016,0.0,0.0,0.002783,0.012245,0.020317,0.005383,0.002444,0.003279,0.057042,0.037968,0.006439,0.000076,0.009843,...,0.0,0.000129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000078,0.0,0.0,0.000026,0.000026,0.000465,0.000465,0.0,0.000000,0.0,0.000000,0.000207,0.0,0.000672,0.0,0.00000,0.000103,0.000284,0.000284,0.000078,0.000000,0.0,0.000207,0.000000,0.000026,0.002224,0.000103,0.000000,0.00000,0.000207,0.000284
A81004,0.003557,0.007192,0.012278,0.000534,0.010558,0.012285,0.004161,0.0,0.010103,0.000474,0.005260,0.000475,0.009286,0.014538,0.017638,0.0,0.065608,0.007500,0.000068,0.023072,0.038918,0.065575,0.000315,0.003880,0.001472,0.0,0.000174,0.0,0.0,0.004530,0.009290,0.021715,0.000697,0.000000,0.009379,0.082202,0.034085,0.005774,0.000000,0.001746,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000054,0.000000,0.000324,0.000702,0.0,0.000054,0.0,0.000054,0.000108,0.0,0.000702,0.0,0.00000,0.000000,0.000270,0.000108,0.000054,0.000000,0.0,0.001026,0.000054,0.000000,0.002485,0.000162,0.000054,0.00000,0.000270,0.000162
A81005,0.003776,0.006354,0.011648,0.000519,0.021563,0.017461,0.001220,0.0,0.011310,0.000524,0.005426,0.000474,0.009008,0.016334,0.022888,0.0,0.078637,0.009678,0.000281,0.019904,0.044886,0.083865,0.000566,0.002829,0.002127,0.0,0.000000,0.0,0.0,0.001396,0.016530,0.015983,0.002051,0.000676,0.002326,0.040973,0.032853,0.017803,0.000000,0.004628,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000601,0.000841,0.0,0.000180,0.0,0.000060,0.000000,0.0,0.000601,0.0,0.00006,0.000000,0.000300,0.000180,0.000000,0.000000,0.0,0.000541,0.000361,0.000000,0.002584,0.000361,0.000060,0.00006,0.000240,0.000240
A81006,0.002379,0.007947,0.013156,0.001236,0.011542,0.012784,0.001048,0.0,0.008469,0.000452,0.006002,0.000505,0.009739,0.014476,0.027667,0.0,0.048399,0.008656,0.000522,0.018335,0.067174,0.108763,0.000583,0.004162,0.003989,0.0,0.000046,0.0,0.0,0.001546,0.011491,0.020368,0.003944,0.000123,0.004057,0.056153,0.039640,0.006305,0.000400,0.009002,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000104,0.0,0.0,0.000000,0.000000,0.000415,0.000346,0.0,0.000000,0.0,0.000000,0.000138,0.0,0.000415,0.0,0.00000,0.000000,0.000173,0.000035,0.000000,0.000035,0.0,0.000242,0.000069,0.000000,0.002388,0.000069,0.000000,0.00000,0.

### Write Data to csv

In [0]:
drug_sales_by_sections.to_csv('./data_mod/bnf_data_by_practice_and_sections.csv')

## 1.5 Prepare Practice Data

### Load Data

In [0]:
practice_raw_data = pd.read_csv("./data/Patients grouped by age and gender per Practice - Jul 2018.csv")

### Clean Data

In [0]:
# Get same name Structure like BNF Data
practice_col_name = {"Practice": "Practice Name", "Code.4":"PRACTICE"}
practice_raw_data.rename(columns=practice_col_name, inplace=True)

In [0]:
# Fix wrong formated number data (comma instead of dot)
practice_raw_data["List Size"] = practice_raw_data["List Size"].apply(lambda x: float(x.replace(',','.')))
practice_raw_data["Total List Size"] = practice_raw_data["Total List Size"].apply(lambda x: float(x.replace(',','.')))

In [30]:
# Rename Age Group to Age Group and Gender split Data to columns
practice_raw_data.rename(columns={"Age Group": "Gender and Age Group"}, inplace=True)
practice_raw_data["Gender"] = practice_raw_data["Gender and Age Group"].apply(lambda x:re.findall(r"^[a-zA-Z]*",x)[0])
practice_raw_data["Age Group"] = practice_raw_data["Gender and Age Group"].apply(lambda x:re.findall(r"[0-9\-\+]*$",x)[0])
practice_raw_data

,Region,Code,Local Office,Code.1,Area,Code.2,Comm./Prov.,Code.3,Practice Name,PRACTICE,Total List Size,Gender and Age Group,List Size,Gender,Age Group
0,LONDON,Y56,LONDON LOCAL OFFICE,Q71,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00,ABBEY MEDICAL CENTRE,F82625,240.0,Female 00-04,240.0,Female,00-04
1,LONDON,Y56,LONDON LOCAL OFFICE,Q71,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00,ABBEY MEDICAL CENTRE,F82625,525.0,Female 05-14,525.0,Female,05-14
2,LONDON,Y56,LONDON LOCAL OFFICE,Q71,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00,ABBEY MEDICAL CENTRE,F82625,461.0,Female 15-24,461.0,Female,15-24
3,LONDON,Y56,LONDON LOCAL OFFICE,Q71,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00,ABBEY MEDICAL CENTRE,F82625,577.0,Female 25-34,577.0,Female,25-34
4,LONDON,Y56,LONDON LOCAL OFFICE,Q71,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00,ABBEY MEDICAL CENTRE,F82625,514.0,Female 35-44,514.0,Female,35-44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122773,SOUTH OF ENGLAND,Y57,SOUTH (WESSEX) LOCAL OFFICE,Q70,WESSEX AREA,Q70,WEST HAMPSHIRE CCG,11A00,WISTARIA & MILFORD SURGERIES,J82139,646.0,Male 35-44,646.0,Male,35-44
122774,SOUTH OF ENGLAND,Y57,SOUTH (WESSEX) LOCAL OFFICE,Q70,WESSEX AREA,Q70,WEST HAMPSHIRE CCG,11A00,WISTARIA & MILFORD SURGERIES,J82139,922.0,Male 45-54,922.0,Male,45-54
122775,SOUTH OF ENGLAND,Y57,SOUTH (WESSEX) LOCAL OFFICE,Q70,WESSEX AREA,Q70,WEST HAMPSHIRE CCG,11A00,WISTARIA & MILFORD SURGERIES,J82139,1086.0,Male 55-64,1086.0,Male,55-64
122776,SOUTH OF ENGLAND,Y57,SOUTH (WESSEX) LOCAL OFFICE,Q70,WESSEX AREA,Q70,WEST HAMPSHIRE CCG,11A00,WISTARIA & MILFORD SURGERIES,J82139,1300.0,Male 65-74,1300.0,Male,65-74


In [31]:
practice_raw_data.columns

Index(['Region', 'Code', 'Local Office', 'Code.1', 'Area', 'Code.2',
       'Comm./Prov.', 'Code.3', 'Practice Name', 'PRACTICE', 'Total List Size',
       'Gender and Age Group', 'List Size', 'Gender', 'Age Group'],
      dtype='object')

### Create Practice Location File

In [0]:
practice_location = practice_raw_data[['PRACTICE', 'Practice Name', 'Region', 'Code', 'Local Office', 'Code.1', 'Area', 'Code.2',
       'Comm./Prov.', 'Code.3']].drop_duplicates()

In [33]:
practice_location

,PRACTICE,Practice Name,Region,Code,Local Office,Code.1,Area,Code.2,Comm./Prov.,Code.3
0,F82625,ABBEY MEDICAL CENTRE,LONDON,Y56,LONDON LOCAL OFFICE,Q71,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00
18,F82018,BARKING MEDICAL GROUP PRACTICE,LONDON,Y56,LONDON LOCAL OFFICE,Q71,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00
36,F86040,DR A ARIF,LONDON,Y56,LONDON LOCAL OFFICE,Q71,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00
54,F82677,DR A MOGHAL'S PRACTICE,LONDON,Y56,LONDON LOCAL OFFICE,Q71,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00
72,F82650,DR AA ANSARI'S PRACTICE,LONDON,Y56,LONDON LOCAL OFFICE,Q71,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00
...,...,...,...,...,...,...,...,...,...,...
122688,J82008,WEST END SURGERY,SOUTH OF ENGLAND,Y57,SOUTH (WESSEX) LOCAL OFFICE,Q70,WESSEX AREA,Q70,WEST HAMPSHIRE CCG,11A00
122706,J82036,WEST MEON SURGERY,SOUTH OF ENGLAND,Y57,SOUTH (WESSEX) LOCAL OFFICE,Q70,WESSEX AREA,Q70,WEST HAMPSHIRE CCG,11A00
122724,J82214,WHITCHURCH SURGERY,SOUTH OF ENGLAND,Y57,SOUTH (WESSEX) LOCAL OFFICE,Q70,WESSEX AREA,Q70,WEST HAMPSHIRE CCG,11A00
122742,J82034,WICKHAM SURGERY,SOUTH OF ENGLAND,Y57,SOUTH (WESSEX) LOCAL OFFICE,Q70,WESSEX AREA,Q70,WEST HAMPSHIRE CCG,11A00


In [0]:
practice_location.to_csv("./data_mod/location_of_practice.csv")

### Create Practice Patient File

In [35]:
practice_raw_data.columns

Index(['Region', 'Code', 'Local Office', 'Code.1', 'Area', 'Code.2',
       'Comm./Prov.', 'Code.3', 'Practice Name', 'PRACTICE', 'Total List Size',
       'Gender and Age Group', 'List Size', 'Gender', 'Age Group'],
      dtype='object')

In [36]:
patient_by_practice_aggregated = practice_raw_data[[\
                          "PRACTICE", "Gender and Age Group", "Age Group", "Gender", "List Size"]]\
                          .groupby(["PRACTICE","Gender and Age Group", "Gender", "Age Group"]).agg("sum")
patient_by_practice_aggregated

List Size
PRACTICE Gender and Age Group Gender Age Group           
A81001   Female 00-04         Female 00-04          101.0
         Female 05-14         Female 05-14          258.0
         Female 15-24         Female 15-24          222.0
         Female 25-34         Female 25-34          261.0
         Female 35-44         Female 35-44          217.0
...                                                   ...
Y06507   Male 35-44           Male   35-44          401.0
         Male 45-54           Male   45-54          396.0
         Male 55-64           Male   55-64          400.0
         Male 65-74           Male   65-74          266.0
         Male 75+             Male   75+            190.0

[122778 rows x 1 columns]

### Normalize Data

In [0]:
# Total Number of patients per Practice
patient_by_practice_summed = practice_raw_data[["PRACTICE", "List Size"]].groupby(["PRACTICE"]).agg("sum")

In [38]:
practice_raw_data[["PRACTICE", "Total List Size"]]

,PRACTICE,Total List Size
0,F82625,240.0
1,F82625,525.0
2,F82625,461.0
3,F82625,577.0
4,F82625,514.0
...,...,...
122773,J82139,646.0
122774,J82139,922.0
122775,J82139,1086.0
122776,J82139,1300.0


In [39]:
patient_by_practice_normalized = patient_by_practice_aggregated.copy()
patient_by_practice_normalized["List Size"] = patient_by_practice_aggregated["List Size"] / patient_by_practice_summed["List Size"]

patient_by_practice_normalized.reset_index(inplace=True)
patient_by_practice_normalized

,PRACTICE,Gender and Age Group,Gender,Age Group,List Size
0,A81001,Female 00-04,Female,00-04,0.024509
1,A81001,Female 05-14,Female,05-14,0.062606
2,A81001,Female 15-24,Female,15-24,0.053870
3,A81001,Female 25-34,Female,25-34,0.063334
4,A81001,Female 35-44,Female,35-44,0.052657
...,...,...,...,...,...
122773,Y06507,Male 35-44,Male,35-44,0.072487
122774,Y06507,Male 45-54,Male,45-54,0.071584
122775,Y06507,Male 55-64,Male,55-64,0.072307
122776,Y06507,Male 65-74,Male,65-74,0.048084


### Gender and Age Group to columns

In [49]:
# Create pandas DF with Gender in Columns 
patient_by_practice_normalized_gender = patient_by_practice_normalized[["PRACTICE","Gender", "List Size"]].groupby(["PRACTICE", "Gender"]).agg("sum")
patient_by_practice_normalized_gender.reset_index(inplace=True)

patient_by_practice_pivot_gender = patient_by_practice_normalized_gender.pivot(index= 'PRACTICE', columns='Gender', values= 'List Size')
patient_by_practice_pivot_gender.fillna(0, inplace=True)
patient_by_practice_pivot_gender


Gender,Female,Male
PRACTICE,,
A81001,0.506673,0.493327
A81002,0.507403,0.492597
A81004,0.483163,0.516837
A81005,0.515166,0.484834
A81006,0.503553,0.496447
...,...,...
Y06389,0.253731,0.746269
Y06443,0.507468,0.492532
Y06487,0.507079,0.492921


In [50]:
# Create pandas DF with Age Group in Columns 
patient_by_practice_normalized_age = patient_by_practice_normalized[["PRACTICE","Age Group", "List Size"]].groupby(["PRACTICE", "Age Group"]).agg("sum")
patient_by_practice_normalized_age.reset_index(inplace=True)

patient_by_practice_pivot_age = patient_by_practice_normalized_age.pivot(index= 'PRACTICE', columns='Age Group', values= 'List Size')
patient_by_practice_pivot_age.fillna(0, inplace=True)
patient_by_practice_pivot_age


Age Group,00-04,05-14,15-24,25-34,35-44,45-54,55-64,65-74,75+
PRACTICE,,,,,,,,,
A81001,0.049988,0.120844,0.104829,0.125212,0.117205,0.124970,0.123999,0.114778,0.118175
A81002,0.047379,0.109668,0.099226,0.133306,0.106291,0.133254,0.152475,0.115902,0.102499
A81004,0.057153,0.111394,0.109756,0.160903,0.124135,0.123771,0.121223,0.100746,0.090917
A81005,0.041631,0.108437,0.080560,0.098244,0.112980,0.132506,0.133120,0.160629,0.131892
A81006,0.055695,0.126751,0.108344,0.139338,0.121540,0.128375,0.139135,0.104690,0.076132
...,...,...,...,...,...,...,...,...,...
Y06389,0.000000,0.000000,0.059701,0.194030,0.238806,0.343284,0.104478,0.044776,0.014925
Y06443,0.056499,0.120082,0.099343,0.133652,0.124691,0.142528,0.124093,0.102671,0.096441
Y06487,0.004719,0.006293,0.535396,0.321972,0.085999,0.029890,0.012585,0.002622,0.000524


In [51]:
# Create pandas DF with Gender and Age Group in Columns 
patient_by_practice_pivot_age_and_gender_group = patient_by_practice_normalized[["PRACTICE","Gender and Age Group", "List Size"]]\
                                                  .pivot(index= 'PRACTICE', columns='Gender and Age Group', values= 'List Size')
patient_by_practice_pivot_age_and_gender_group.fillna(0, inplace=True)
patient_by_practice_pivot_age_and_gender_group



Gender and Age Group,Female 00-04,Female 05-14,Female 15-24,Female 25-34,Female 35-44,Female 45-54,Female 55-64,Female 65-74,Female 75+,Male 00-04,Male 05-14,Male 15-24,Male 25-34,Male 35-44,Male 45-54,Male 55-64,Male 65-74,Male 75+
PRACTICE,,,,,,,,,,,,,,,,,,
A81001,0.024509,0.062606,0.053870,0.063334,0.052657,0.061636,0.063819,0.057753,0.066489,0.025479,0.058238,0.050959,0.061878,0.064547,0.063334,0.060180,0.057025,0.051686
A81002,0.025144,0.054860,0.047795,0.066237,0.051327,0.066185,0.075952,0.059847,0.060055,0.022235,0.054808,0.051431,0.067068,0.054964,0.067068,0.076523,0.056055,0.042444
A81004,0.027120,0.055060,0.051875,0.072716,0.057153,0.057608,0.057790,0.052330,0.051511,0.030033,0.056334,0.057881,0.088187,0.066982,0.066163,0.063433,0.048416,0.039407
A81005,0.020877,0.054280,0.040280,0.048876,0.057350,0.066437,0.069999,0.085841,0.071227,0.020754,0.054157,0.040280,0.049368,0.055631,0.066069,0.063122,0.074788,0.060666
A81006,0.026257,0.061108,0.053055,0.069500,0.060702,0.062665,0.070312,0.054544,0.045408,0.029438,0.065643,0.055289,0.069838,0.060838,0.065710,0.068823,0.050145,0.030723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y06389,0.000000,0.000000,0.029851,0.059701,0.044776,0.089552,0.014925,0.014925,0.000000,0.000000,0.000000,0.029851,0.134328,0.194030,0.253731,0.089552,0.029851,0.014925
Y06443,0.027396,0.059145,0.049074,0.068448,0.063412,0.071264,0.062729,0.052317,0.053683,0.029103,0.060937,0.050269,0.065204,0.061278,0.071264,0.061364,0.050354,0.042758
Y06487,0.001573,0.002622,0.317777,0.141059,0.028317,0.009963,0.004719,0.001049,0.000000,0.003146,0.003671,0.217619,0.180912,0.057682,0.019927,0.007866,0.001573,0.000524


In [52]:
# Merge all together again
# patient_by_practice_pivot_gender
# patient_by_practice_pivot_age
# patient_by_practice_pivot_age_and_gender_group

patient_by_practice_pivot = patient_by_practice_pivot_gender.merge(patient_by_practice_pivot_age, left_index=True, right_index=True)
patient_by_practice_pivot = patient_by_practice_pivot.merge(patient_by_practice_pivot_age_and_gender_group, left_index=True, right_index=True)
patient_by_practice_pivot

,Female,Male,00-04,05-14,15-24,25-34,35-44,45-54,55-64,65-74,75+,Female 00-04,Female 05-14,Female 15-24,Female 25-34,Female 35-44,Female 45-54,Female 55-64,Female 65-74,Female 75+,Male 00-04,Male 05-14,Male 15-24,Male 25-34,Male 35-44,Male 45-54,Male 55-64,Male 65-74,Male 75+
PRACTICE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A81001,0.506673,0.493327,0.049988,0.120844,0.104829,0.125212,0.117205,0.124970,0.123999,0.114778,0.118175,0.024509,0.062606,0.053870,0.063334,0.052657,0.061636,0.063819,0.057753,0.066489,0.025479,0.058238,0.050959,0.061878,0.064547,0.063334,0.060180,0.057025,0.051686
A81002,0.507403,0.492597,0.047379,0.109668,0.099226,0.133306,0.106291,0.133254,0.152475,0.115902,0.102499,0.025144,0.054860,0.047795,0.066237,0.051327,0.066185,0.075952,0.059847,0.060055,0.022235,0.054808,0.051431,0.067068,0.054964,0.067068,0.076523,0.056055,0.042444
A81004,0.483163,0.516837,0.057153,0.111394,0.109756,0.160903,0.124135,0.123771,0.121223,0.100746,0.090917,0.027120,0.055060,0.051875,0.072716,0.057153,0.057608,0.057790,0.052330,0.051511,0.030033,0.056334,0.057881,0.088187,0.066982,0.066163,0.063433,0.048416,0.039407
A81005,0.515166,0.484834,0.041631,0.108437,0.080560,0.098244,0.112980,0.132506,0.133120,0.160629,0.131892,0.020877,0.054280,0.040280,0.048876,0.057350,0.066437,0.069999,0.085841,0.071227,0.020754,0.054157,0.040280,0.049368,0.055631,0.066069,0.063122,0.074788,0.060666
A81006,0.503553,0.496447,0.055695,0.126751,0.108344,0.139338,0.121540,0.128375,0.139135,0.104690,0.076132,0.026257,0.061108,0.053055,0.069500,0.060702,0.062665,0.070312,0.054544,0.045408,0.029438,0.065643,0.055289,0.069838,0.060838,0.065710,0.068823,0.050145,0.030723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y06389,0.253731,0.746269,0.000000,0.000000,0.059701,0.194030,0.238806,0.343284,0.104478,0.044776,0.014925,0.000000,0.000000,0.029851,0.059701,0.044776,0.089552,0.014925,0.014925,0.000000,0.000000,0.000000,0.029851,0.134328,0.194030,0.253731,0.089552,0.029851,0.014925
Y06443,0.507468,0.492532,0.056499,0.120082,0.099343,0.133652,0.124691,0.142528,0.124093,0.102671,0.096441,0.027396,0.059145,0.049074,0.068448,0.063412,0.071264,0.062729,0.052317,0.053683,0.029103,0.060937,0.050269,0.065204,0.061278,0.071264,0.061364,0.050354,0.042758
Y06487,0.507079,0.492921,0.004719,0.006293,0.535396,0.321972,0.085999,0.029890,0.012585,0.002622,0.000524,0.001573,0.002622,0.317777,0.141059,0.028317,0.009963,0.004719,0.001049,0.000000,0.003146,0.003671,0.217619,0.180912,0.057682,0.019927,0.007866,0.001573,0.000524


### Add ITEMS and NIC Data (total by Practice)

In [53]:
patients_and_drug_sales_by_practice = patient_by_practice_pivot.merge(drug_sales_sumemd_by_practice, left_index=True ,right_index=True)
patients_and_drug_sales_by_practice

,Female,Male,00-04,05-14,15-24,25-34,35-44,45-54,55-64,65-74,75+,Female 00-04,Female 05-14,Female 15-24,Female 25-34,Female 35-44,Female 45-54,Female 55-64,Female 65-74,Female 75+,Male 00-04,Male 05-14,Male 15-24,Male 25-34,Male 35-44,Male 45-54,Male 55-64,Male 65-74,Male 75+,ITEMS,NIC
PRACTICE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A81001,0.506673,0.493327,0.049988,0.120844,0.104829,0.125212,0.117205,0.124970,0.123999,0.114778,0.118175,0.024509,0.062606,0.053870,0.063334,0.052657,0.061636,0.063819,0.057753,0.066489,0.025479,0.058238,0.050959,0.061878,0.064547,0.063334,0.060180,0.057025,0.051686,8005,58349.07
A81002,0.507403,0.492597,0.047379,0.109668,0.099226,0.133306,0.106291,0.133254,0.152475,0.115902,0.102499,0.025144,0.054860,0.047795,0.066237,0.051327,0.066185,0.075952,0.059847,0.060055,0.022235,0.054808,0.051431,0.067068,0.054964,0.067068,0.076523,0.056055,0.042444,38669,291568.90
A81004,0.483163,0.516837,0.057153,0.111394,0.109756,0.160903,0.124135,0.123771,0.121223,0.100746,0.090917,0.027120,0.055060,0.051875,0.072716,0.057153,0.057608,0.057790,0.052330,0.051511,0.030033,0.056334,0.057881,0.088187,0.066982,0.066163,0.063433,0.048416,0.039407,18514,156655.28
A81005,0.515166,0.484834,0.041631,0.108437,0.080560,0.098244,0.112980,0.132506,0.133120,0.160629,0.131892,0.020877,0.054280,0.040280,0.048876,0.057350,0.066437,0.069999,0.085841,0.071227,0.020754,0.054157,0.040280,0.049368,0.055631,0.066069,0.063122,0.074788,0.060666,16639,110988.46
A81006,0.503553,0.496447,0.055695,0.126751,0.108344,0.139338,0.121540,0.128375,0.139135,0.104690,0.076132,0.026257,0.061108,0.053055,0.069500,0.060702,0.062665,0.070312,0.054544,0.045408,0.029438,0.065643,0.055289,0.069838,0.060838,0.065710,0.068823,0.050145,0.030723,28890,203691.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y05788,0.120192,0.879808,0.000000,0.000000,0.057692,0.312500,0.269231,0.264423,0.076923,0.014423,0.004808,0.000000,0.000000,0.000000,0.038462,0.062500,0.014423,0.004808,0.000000,0.000000,0.000000,0.000000,0.057692,0.274038,0.206731,0.250000,0.072115,0.014423,0.004808,706,6129.84
Y05826,0.470529,0.529471,0.097956,0.211282,0.147441,0.196926,0.181050,0.096099,0.040534,0.016720,0.011991,0.050836,0.099983,0.077352,0.088667,0.080730,0.038676,0.019422,0.007431,0.007431,0.047120,0.111299,0.070090,0.108259,0.100321,0.057423,0.021111,0.009289,0.004560,3803,33610.50
Y06007,0.527949,0.472051,0.060773,0.147546,0.120572,0.150796,0.128209,0.136009,0.113260,0.076373,0.066461,0.033962,0.067111,0.059149,0.088073,0.068573,0.077998,0.056874,0.037049,0.039162,0.026812,0.080435,0.061423,0.062723,0.059636,0.058011,0.056386,0.039324,0.027299,11884,82423.82


### Write Data to CSV

In [0]:
patients_and_drug_sales_by_practice.to_csv("./data_mod/patients_and_drug_sales_by_practice_and_demographics.csv")

## 1.6 Create Big Files with Patient Infos and Drug Sales per Chapter and Section

### Chapter Data

In [61]:
patients_and_drug_sales_by_practice_per_chapter = patients_and_drug_sales_by_practice.merge(drug_sales_by_chapters, right_index=True, left_index=True)
patients_and_drug_sales_by_practice_per_chapter

,Female,Male,00-04,05-14,15-24,25-34,35-44,45-54,55-64,65-74,75+,Female 00-04,Female 05-14,Female 15-24,Female 25-34,Female 35-44,Female 45-54,Female 55-64,Female 65-74,Female 75+,Male 00-04,Male 05-14,Male 15-24,Male 25-34,Male 35-44,Male 45-54,Male 55-64,Male 65-74,Male 75+,ITEMS,NIC,NIC_1,NIC_2,NIC_3,NIC_4,NIC_5,NIC_6,NIC_7,NIC_8,NIC_9,NIC_10,NIC_11,NIC_12,NIC_13,NIC_14,NIC_15,NIC_18,NIC_19,NIC_20,NIC_21,NIC_22,NIC_23,ITEMS_1,ITEMS_2,ITEMS_3,ITEMS_4,ITEMS_5,ITEMS_6,ITEMS_7,ITEMS_8,ITEMS_9,ITEMS_10,ITEMS_11,ITEMS_12,ITEMS_13,ITEMS_14,ITEMS_15,ITEMS_18,ITEMS_19,ITEMS_20,ITEMS_21,ITEMS_22,ITEMS_23
PRACTICE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A81001,0.506673,0.493327,0.049988,0.120844,0.104829,0.125212,0.117205,0.124970,0.123999,0.114778,0.118175,0.024509,0.062606,0.053870,0.063334,0.052657,0.061636,0.063819,0.057753,0.066489,0.025479,0.058238,0.050959,0.061878,0.064547,0.063334,0.060180,0.057025,0.051686,8005,58349.07,0.056223,0.157726,0.142735,0.167511,0.016819,0.156060,0.037295,0.017994,0.074164,0.011891,0.027607,0.007233,0.037510,0.009665,0.007483,0.0,0.001373,0.015130,0.031614,0.004434,0.019533,0.086321,0.249219,0.073204,0.247345,0.030106,0.096190,0.020862,0.004997,0.061212,0.022986,0.019738,0.008245,0.041099,0.004747,0.000750,0.0,0.001124,0.007995,0.018863,0.001374,0.003623
A81002,0.507403,0.492597,0.047379,0.109668,0.099226,0.133306,0.106291,0.133254,0.152475,0.115902,0.102499,0.025144,0.054860,0.047795,0.066237,0.051327,0.066185,0.075952,0.059847,0.060055,0.022235,0.054808,0.051431,0.067068,0.054964,0.067068,0.076523,0.056055,0.042444,38669,291568.90,0.061945,0.145536,0.161891,0.157818,0.018375,0.149729,0.034502,0.021213,0.077802,0.014817,0.022739,0.006790,0.033317,0.004202,0.003055,0.0,0.001670,0.012934,0.037903,0.003074,0.030689,0.094262,0.293181,0.078331,0.209160,0.027412,0.089865,0.024542,0.003491,0.062686,0.029300,0.017818,0.008715,0.027878,0.002198,0.000905,0.0,0.000905,0.005120,0.018490,0.001060,0.004681
A81004,0.483163,0.516837,0.057153,0.111394,0.109756,0.160903,0.124135,0.123771,0.121223,0.100746,0.090917,0.027120,0.055060,0.051875,0.072716,0.057153,0.057608,0.057790,0.052330,0.051511,0.030033,0.056334,0.057881,0.088187,0.066982,0.066163,0.063433,0.048416,0.039407,18514,156655.28,0.060667,0.143918,0.110334,0.169418,0.031567,0.148483,0.042506,0.013335,0.077945,0.012791,0.020668,0.006895,0.040052,0.000800,0.003071,0.0,0.001653,0.028635,0.043309,0.003559,0.040393,0.086421,0.288862,0.065302,0.211678,0.032192,0.108945,0.027115,0.004267,0.040834,0.029977,0.018797,0.010263,0.036621,0.000486,0.000756,0.0,0.001188,0.007994,0.021659,0.001188,0.005455
A81005,0.515166,0.484834,0.041631,0.108437,0.080560,0.098244,0.112980,0.132506,0.133120,0.160629,0.131892,0.020877,0.054280,0.040280,0.048876,0.057350,0.066437,0.069999,0.085841,0.071227,0.020754,0.054157,0.040280,0.049368,0.055631,0.066069,0.063122,0.074788,0.060666,16639,110988.46,0.073851,0.163154,0.134273,0.135219,0.016623,0.130996,0.047607,0.025108,0.055085,0.010108,0.021823,0.008470,0.035835,0.003579,0.003217,0.0,0.001429,0.017893,0.064823,0.006161,0.044748,0.098744,0.345634,0.062744,0.161849,0.023619,0.108961,0.033415,0.005770,0.042551,0.025242,0.018571,0.011659,0.025422,0.001683,0.000901,0.0,0.000841,0.003846,0.021275,0.001683,0.005589
A81006,0.503553,0.496447,0.055695,0.126751,0.108344,0.139338,0.121540,0.128375,0.139135,0.104690,0.076132,0.026257,0.061108,0.053055,0.069500,0.060702,0.062665,0.070312,0.054544,0.045408,0.029438,0.065643,0.055289,0.069838,0.060838,0.065710,0.068823,0.050145,0.030723,28890,203691.89,0.058561,0.134753,0.184717,0.153028,0.022751,0.139594,0.035716,0.031853,0.071647,0.015066,0.019302,0.007284,0.034108,0.001410,0.005415,0.0,0.003071,0.020939,0.040715,0.002512,0.017556,0.094323,0.309311,0.088266,0.204500,0.026653,0.096435,0.022015,0.003565,0.042575,0.027449,0.013776,0.008169,0.028072,0.000658,0.001142,0.0,0.000935,0.007269,0.020249,0.000865,0.003773
...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [0]:
patients_and_drug_sales_by_practice_per_chapter.to_csv("./data_mod/patients_and_drug_sales_by_practice_per_demographics_and_chapters.csv")

### Section Data

In [63]:
patients_and_drug_sales_by_practice_per_section = patients_and_drug_sales_by_practice.merge(drug_sales_by_sections, right_index=True, left_index=True)
patients_and_drug_sales_by_practice_per_section

,Female,Male,00-04,05-14,15-24,25-34,35-44,45-54,55-64,65-74,75+,Female 00-04,Female 05-14,Female 15-24,Female 25-34,Female 35-44,Female 45-54,Female 55-64,Female 65-74,Female 75+,Male 00-04,Male 05-14,Male 15-24,Male 25-34,Male 35-44,Male 45-54,Male 55-64,Male 65-74,Male 75+,ITEMS,NIC,NIC_101,NIC_102,NIC_103,NIC_104,NIC_105,NIC_106,NIC_107,NIC_108,NIC_109,...,ITEMS_2140,ITEMS_2141,ITEMS_2142,ITEMS_2143,ITEMS_2146,ITEMS_2147,ITEMS_2148,ITEMS_2149,ITEMS_2202,ITEMS_2205,ITEMS_2210,ITEMS_2215,ITEMS_2230,ITEMS_2240,ITEMS_2250,ITEMS_2260,ITEMS_2270,ITEMS_2280,ITEMS_2285,ITEMS_2290,ITEMS_2305,ITEMS_2310,ITEMS_2315,ITEMS_2320,ITEMS_2325,ITEMS_2330,ITEMS_2335,ITEMS_2345,ITEMS_2346,ITEMS_2350,ITEMS_2355,ITEMS_2360,ITEMS_2370,ITEMS_2375,ITEMS_2380,ITEMS_2385,ITEMS_2390,ITEMS_2392,ITEMS_2394,ITEMS_2396
PRACTICE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A81001,0.506673,0.493327,0.049988,0.120844,0.104829,0.125212,0.117205,0.124970,0.123999,0.114778,0.118175,0.024509,0.062606,0.053870,0.063334,0.052657,0.061636,0.063819,0.057753,0.066489,0.025479,0.058238,0.050959,0.061878,0.064547,0.063334,0.060180,0.057025,0.051686,8005,58349.07,0.003724,0.003433,0.013764,0.000632,0.014294,0.012271,0.000506,0.0,0.007597,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000125,0.000125,0.000625,0.000500,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000500,0.0,0.00000,0.000000,0.000375,0.000125,0.000000,0.000000,0.0,0.000250,0.000000,0.000000,0.002374,0.000000,0.000000,0.00000,0.000000,0.000000
A81002,0.507403,0.492597,0.047379,0.109668,0.099226,0.133306,0.106291,0.133254,0.152475,0.115902,0.102499,0.025144,0.054860,0.047795,0.066237,0.051327,0.066185,0.075952,0.059847,0.060055,0.022235,0.054808,0.051431,0.067068,0.054964,0.067068,0.076523,0.056055,0.042444,38669,291568.90,0.002171,0.004811,0.012788,0.000500,0.013234,0.014250,0.000717,0.0,0.013473,...,0.0,0.000129,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000078,0.0,0.0,0.000026,0.000026,0.000465,0.000465,0.0,0.000000,0.0,0.000000,0.000207,0.0,0.000672,0.0,0.00000,0.000103,0.000284,0.000284,0.000078,0.000000,0.0,0.000207,0.000000,0.000026,0.002224,0.000103,0.000000,0.00000,0.000207,0.000284
A81004,0.483163,0.516837,0.057153,0.111394,0.109756,0.160903,0.124135,0.123771,0.121223,0.100746,0.090917,0.027120,0.055060,0.051875,0.072716,0.057153,0.057608,0.057790,0.052330,0.051511,0.030033,0.056334,0.057881,0.088187,0.066982,0.066163,0.063433,0.048416,0.039407,18514,156655.28,0.003557,0.007192,0.012278,0.000534,0.010558,0.012285,0.004161,0.0,0.010103,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000054,0.000000,0.000324,0.000702,0.0,0.000054,0.0,0.000054,0.000108,0.0,0.000702,0.0,0.00000,0.000000,0.000270,0.000108,0.000054,0.000000,0.0,0.001026,0.000054,0.000000,0.002485,0.000162,0.000054,0.00000,0.000270,0.000162
A81005,0.515166,0.484834,0.041631,0.108437,0.080560,0.098244,0.112980,0.132506,0.133120,0.160629,0.131892,0.020877,0.054280,0.040280,0.048876,0.057350,0.066437,0.069999,0.085841,0.071227,0.020754,0.054157,0.040280,0.049368,0.055631,0.066069,0.063122,0.074788,0.060666,16639,110988.46,0.003776,0.006354,0.011648,0.000519,0.021563,0.017461,0.001220,0.0,0.011310,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000601,0.000841,0.0,0.000180,0.0,0.000060,0.000000,0.0,0.000601,0.0,0.00006,0.000000,0.000300,0.000180,0.000000,0.000000,0.0,0.000541,0.000361,0.000000,0.002584,0.000361,0.000060,0.00006,0.000240,0.000240
A81006,0.503553,0.496447,0.055695,0.126751,0.108344,0.139338,0.121540,0.128375,0.139135,0.104690,0.076132,0.026257,0.061108,0.053055,0.069500,0.060702,0.062665,0.070312,0.054544,0.045408,0.029438,0.065643,0.055289,0.069838,0.060838,0.065710,0.068823,0.050145,0.030723,28890,203691.89,0.002379,0.007947,0.013156,0.001236,0.011542,0.012784,0.001048,0.0,0.008469,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000104,0.0,0.0,0.000000,0.000000,0.000415,0.000346,0.0,0.000000,0.0,0.000000,0.000138,0.0

In [0]:
patients_and_drug_sales_by_practice_per_section.to_csv("./data_mod/patients_and_drug_sales_by_practice_per_demographics_and_sections.csv")